In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import math
import re
import random
import string
from tqdm import tqdm
import time
from collections import Counter

import nltk
import torch
import spacy
from sklearn.preprocessing import binarize

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split, Subset

import transformers
from transformers import BertTokenizer, BertModel
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup, get_constant_schedule_with_warmup
import tokenizers
from tokenizers import BertWordPieceTokenizer

In [ ]:
train_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
ss_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

train_df.dropna(inplace=True)
train_df.reset_index(drop=True, inplace=True)
train_df

In [ ]:
with open('../input/bert-base-uncased/config.json', 'r') as f:
    bert_config = f.read()
bert_config

In [ ]:
# print('Loading BERT tokenizer...')
# PRETRAINED_TOKENIZER = '../input/bert-base-uncased/vocab.txt'
# tokenizer = tokenizers.BertWordPieceTokenizer(
#         PRETRAINED_TOKENIZER, 
#         lowercase=True
#     )

In [ ]:
class Config:
    MAX_LEN = 128
    TRAIN_BATCH_SIZE = 32
    VALID_BATCH_SIZE = 16
    BERT_PATH = "../input/bert-base-uncased/"
    MODEL_PATH = "pytorch_model.bin"
    TRAINING_FILE = "../input/tweet-train-folds/train_folds.csv"
    PRETRAINED_TOKENIZER = f"{BERT_PATH}/vocab.txt"
    TOKENIZER = tokenizers.BertWordPieceTokenizer(
        PRETRAINED_TOKENIZER, 
        lowercase=True
    )
    PRETRAINED_MODEL = f"{BERT_PATH}/{MODEL_PATH}"
cfg = Config()

### Set folds

In [ ]:
n_folds = 5
fold_size = train_df.shape[0] // n_folds
print(f'Train size={train_df.shape[0]}, fold size={fold_size}')
free_inds = set(range(train_df.shape[0]))
for i in range(n_folds - 1):
    print(i)
    fold_inds = np.random.choice(list(free_inds), fold_size, replace=False)
    train_df.loc[fold_inds, 'fold'] = i
    free_inds -= set(fold_inds)
fold_inds = list(free_inds)
train_df.loc[fold_inds, 'fold'] = n_folds - 1
train_df['fold'] = train_df['fold'].astype(int)
df = train_df.groupby('fold').count()
df

In [ ]:
SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
# def seed_everything(seed_value):
#     random.seed(seed_value)
#     np.random.seed(seed_value)
#     torch.manual_seed(seed_value)
#     os.environ['PYTHONHASHSEED'] = str(seed_value)
    
#     if torch.cuda.is_available(): 
#         torch.cuda.manual_seed(seed_value)
#         torch.cuda.manual_seed_all(seed_value)
#         torch.backends.cudnn.deterministic = True
#         torch.backends.cudnn.benchmark = True

# seed = 1234
# seed_everything(seed)

Dataset

In [ ]:
class TweetDataset:
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.selected_text = 'selected_text' in df.columns
        self.max_len = max_len
        
    def __len__(self):
        return len(self.df)
    
    def _prepare_data(self, text, selected_text, sentiment):
        len_selected = len(selected_text)
#         print(f'text={text}')
#         print(f'selected_text={selected_text}')
#         print(f'len_selected={len_selected}')
       
        start_pos = None
        end_pos = None
        for i, txt in enumerate(text):
            if text[i: i + len_selected] == selected_text:
                start_pos = i
                end_pos = i + len_selected - 1
                break
#         print(f'start_pos={start_pos}')
#         print(f'end_pos={end_pos}')
        
        char_targets = [0] * len(text)
        if start_pos != None and end_pos != None:
            for ct in range(start_pos, end_pos + 1):
                char_targets[ct] = 1
#         print(f'char_targets={char_targets}')
        
        text_enc = self.tokenizer.encode(text)
#         print(f'text_enc={text_enc.tokens}')
        input_ids_orig = text_enc.ids[1:-1]
        CLS_ID = text_enc.ids[0]
        SEP_ID = text_enc.ids[-1]
#         print(f'input_ids={text_enc.ids}')
#         print(f'input_ids_orig={input_ids_orig}')
        text_offsets = text_enc.offsets[1:-1]
#         print(f'text_offsets={text_offsets}')
        
        target_idx = []
        for j, (offset1, offset2) in enumerate(text_offsets):
            if sum(char_targets[offset1: offset2]) > 0:
                target_idx.append(j)
#         print(f'target_idx={target_idx}')

        targets_start = target_idx[0]
        targets_end = target_idx[-1]
#         print(f'targets_start={targets_start}')
#         print(f'targets_end={targets_end}')
                
        sentiment_id = self.tokenizer.encode(sentiment).ids[1]
#         print(f'sentiment_id={sentiment_id}')

        input_ids = [CLS_ID] + [sentiment_id] + [SEP_ID] + input_ids_orig + [SEP_ID]
        token_type_ids = [0, 0, 0] + [1] * (len(input_ids_orig) + 1)
        mask = [1] * len(token_type_ids)
        text_offsets = [(0, 0)] * 3 + text_offsets + [(0, 0)]
        targets_start += 3
        targets_end += 3
        
        padding_length = self.max_len - len(input_ids)
        if padding_length > 0:
            input_ids = input_ids + ([0] * padding_length)
            mask = mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
            text_offsets = text_offsets + ([(0, 0)] * padding_length)

        return {
            'ids': input_ids,
            'mask': mask,
            'token_type_ids': token_type_ids,
            'targets_start': targets_start,
            'targets_end': targets_end,
            'orig_text': text,
            'orig_selected': selected_text,
            'sentiment': sentiment,
            'offsets': text_offsets
        }
        
        return text_enc

    def __getitem__(self, item):
        try:
            sel_text = self.df.iloc[item].selected_text if self.selected_text else ''
            data = self._prepare_data(
                self.df.iloc[item].text, 
                sel_text, 
                self.df.iloc[item].sentiment
            )
        except KeyError:
            print(f'Bad key: {item}')
            print(f'Current index: {self.df.index.values}')
            raise
                  
        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_text': data["orig_text"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }
    
    def __len__(self):
        return len(self.df)

In [ ]:
# train = TweetDataset(train_df, cfg.TOKENIZER, cfg.MAX_LEN)
# itr = iter(train)
# next(itr)

In [ ]:
train_dataset = TweetDataset(train_df, cfg.TOKENIZER, cfg.MAX_LEN)

train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=cfg.TRAIN_BATCH_SIZE,
    num_workers=1
)

for bi, d in enumerate(train_data_loader):
    ids = d["ids"]
    token_type_ids = d["token_type_ids"]
    mask = d["mask"]
    targets_start = d["targets_start"]
    targets_end = d["targets_end"]
    sentiment = d["sentiment"]
    orig_selected = d["orig_selected"]
    orig_text = d["orig_text"]
    targets_start = d["targets_start"]
    targets_end = d["targets_end"]
    offsets = d["offsets"]
    print(orig_selected)    
    print(sentiment)    
    print(targets_start)
    print(targets_end)    
    break

### Using folds instead

In [ ]:
# # split train val
# size = len(train_dataset)
# print(f'dataset size={size}')
# val_size = int(size*0.15)
# print(f'val size={val_size}')
# train_size = size - val_size
# print(f'train size={train_size}')

# train, val = random_split(train_dataset, [train_size, val_size])

### Model

In [ ]:
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, bert_config, bert_path, dropout):
        super(TweetModel, self).__init__(bert_config)
        self.bert = transformers.BertModel.from_pretrained(bert_path, config=bert_config)
        self.drop_out = nn.Dropout(dropout)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        _, _, out = self.bert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )
#         print(f'len(out)={len(out)}')
#         print(f'out[0]={out[0].shape}')
#         print(f'out[-1]={out[-1].shape}')
#         print(f'out[-2]={out[-2].shape}')
        out = torch.cat((out[-1], out[-2]), dim=-1)            # out[i].shape = (batch_size, sequence_length, hidden_size)
        out = self.drop_out(out)
        logits = self.l0(out)
#         print(f'logits.shape={logits.shape}')

        start_logits, end_logits = logits.split(1, dim=-1)
#         print(f'start_logits.shape={start_logits.shape}')
#         print(f'end_logits.shape={end_logits.shape}')

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
#         print(f'start_logits.shape={start_logits.shape}')
#         print(f'end_logits.shape={end_logits.shape}')

        return start_logits, end_logits

### Training functions

In [ ]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    print(f'start_positions={start_positions.shape}')
    print(f'end_positions={start_logits.shape}')
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    return total_loss

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    if (len(a) + len(b) - len(c)) == 0:
        return 0
    return float(len(c)) / (len(a) + len(b) - len(c))

def evaluate(true, pred):
    jac = 0
    for s1, s2 in zip(true, pred):
        jac += jaccard(s1, s2)
    jac /= len(true)
    return jac

def calculate_jaccard_score(
    original_text, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):

    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output = ''
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_text[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    if sentiment_val == "neutral" or len(original_text.split()) < 2:
        filtered_output = original_text

    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output

In [ ]:
def train_fn(data_loader, model, optimizer, device, scheduler=None, max_grad_norm=1.0):
    model.to(device)
    model.train()
    losses = []
    jaccards = []

    tk0 = tqdm(data_loader, total=len(data_loader))
    
    for bi, d in enumerate(tk0):

        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_text = d["orig_text"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)

        model.zero_grad()
        outputs_start, outputs_end = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids,
        )
        loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        if scheduler:
            scheduler.step()

        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
        jaccard_scores = []
        for i, text in enumerate(orig_text):
            selected_text = orig_selected[i]
            text_sentiment = sentiment[i]
            jaccard_score, _ = calculate_jaccard_score(
                original_text=text,
                target_string=selected_text,
                sentiment_val=text_sentiment,
                idx_start=np.argmax(outputs_start[i, :]),
                idx_end=np.argmax(outputs_end[i, :]),
                offsets=offsets[i]
            )
            jaccard_scores.append(jaccard_score)
        jaccards.append(np.mean(jaccard_scores))
        losses.append(loss.item())
        tk0.set_postfix(loss=np.mean(losses), jaccard=np.mean(jaccards))
    return losses, jaccards

In [ ]:
def eval_fn(data_loader, model, device):
    model.to(device)
    model.eval()
    losses = []
    jaccards = []
    
    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader))
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_text = d["orig_text"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"].numpy()

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)

            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
            jaccard_scores = []
            for px, text in enumerate(orig_text):
                selected_text = orig_selected[px]
                text_sentiment = sentiment[px]
                jaccard_score, _ = calculate_jaccard_score(
                    original_text=text,
                    target_string=selected_text,
                    sentiment_val=text_sentiment,
                    idx_start=np.argmax(outputs_start[px, :]),
                    idx_end=np.argmax(outputs_end[px, :]),
                    offsets=offsets[px]
                )
                jaccard_scores.append(jaccard_score)

            jaccards.append(np.mean(jaccard_scores))
            losses.append(loss.item())
            tk0.set_postfix(loss=np.mean(losses), jaccard=np.mean(jaccards))
    
    print(f"Jaccard = {np.mean(jaccards)}")
    return losses, jaccards

### Training parameters

In [ ]:
EPOCHS=2

In [ ]:
def get_model():
    model_config = transformers.BertConfig.from_pretrained(cfg.BERT_PATH)
    model_config.output_hidden_states = True
    model = TweetModel(bert_config=model_config, bert_path=cfg.BERT_PATH, dropout=0.1)
    return model

def get_training_objects():
    device = torch.device("cuda")
    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": 0.001,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 
            "weight_decay": 0.0
        },
    ]
    n_training_steps = math.ceil(len(train_data_loader) / cfg.TRAIN_BATCH_SIZE)*EPOCHS
    n_warmup_steps = int(n_training_steps*0.2)
    print(f'n_training_steps={n_training_steps}')
    print(f'n_warmup_steps={n_warmup_steps}')
    optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, eps=1e-8)
    scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=10)
    #get_linear_schedule_with_warmup(optimizer, num_warmup_steps=10, num_training_steps=n_training_steps)
    return device, optimizer, scheduler

In [ ]:
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME

def save_model(model, model_file):
    # Save a model, configuration that you have fine-tuned

    # If we have a distributed model, save only the encapsulated model
    # (it was wrapped in PyTorch DistributedDataParallel or DataParallel)
    model_to_save = model.module if hasattr(model, 'module') else model

    # If we save using the predefined names, we can load using `from_pretrained`
    output_model_file = model_file + '_' + WEIGHTS_NAME
    output_config_file = model_file + '_' + CONFIG_NAME

    print(output_model_file)
    print(output_config_file)
    torch.save(model_to_save.state_dict(), output_model_file)
    model_to_save.config.to_json_file(output_config_file)

def load_model(model_file):
    # Re-load the saved model and vocabulary
    output_model_file = model_file + '_' + WEIGHTS_NAME
    output_config_file = model_file + '_' + CONFIG_NAME
    config = transformers.BertConfig.from_json_file(output_config_file)
    model = TweetModel(bert_config=config, bert_path=output_model_file, dropout=0.1)
    return model


### Run training

In [ ]:
folds = np.unique(train_df['fold'].values)
models = []
for test_fold in folds:
    val_part_df = train_df[train_df['fold'] == test_fold]
    train_part_df = train_df[train_df['fold'] != test_fold]
    
    val_dataset = TweetDataset(val_part_df, cfg.TOKENIZER, cfg.MAX_LEN)
    train_dataset = TweetDataset(train_part_df, cfg.TOKENIZER, cfg.MAX_LEN)
    
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=cfg.TRAIN_BATCH_SIZE,
        num_workers=1,
        shuffle=True
    )
    val_data_loader = torch.utils.data.DataLoader(
            val_dataset,
            batch_size=cfg.VALID_BATCH_SIZE,
            num_workers=2,
            shuffle=False
        )
    
    # model
    model = get_model()
    
    device, optimizer, scheduler = get_training_objects()
    
    train_losses, train_jaccards = [], []
    val_losses, val_jaccards = [], []
    for epoch in range(EPOCHS):
        print(f'Epoch {epoch}')
        t_losses, t_jaccards = train_fn(train_data_loader, model, optimizer, device, scheduler)
        train_losses.append(np.mean(t_losses))
        train_jaccards.append(np.mean(t_jaccards))
        v_losses, v_jaccards = eval_fn(val_data_loader, model, device)
        val_losses.append(np.mean(v_losses))
        val_jaccards.append(np.mean(v_jaccards))
        print(f"Jaccard Score = {val_jaccards}")
    save_model(model, f'fold{test_fold}')
    models.append(model)

In [ ]:
train_losses, train_jaccards = [], []
val_losses, val_jaccards = [], []

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,4))
ax.set(xlabel='epoch', ylabel='total loss',
    title='loss per epoch')
ax.grid()
train_losses = np.array(train_losses)
ax.plot(train_losses, color='b', label='Train loss')
#ax.plot(val_losses, color='r', label='Validation loss')
ax.legend()

### Generate submission

In [ ]:
# # load models
# models = []
# for fold in folds:
#     models.append(load_model(f'fold{fold}'))
    

In [ ]:
# models[0].eval()
# v_losses, v_jaccards = eval_fn(val_data_loader, models[0], device)

In [ ]:
# # split train val
# size = len(train_dataset)
# print(f'dataset size={size}')
# val_size = int(size*0.15)
# print(f'val size={val_size}')
# train_size = size - val_size
# print(f'train size={train_size}')

# train, val = random_split(train_dataset, [train_size, val_size])

# val_data_loader = torch.utils.data.DataLoader(
#             val,
#             batch_size=cfg.VALID_BATCH_SIZE,
#             num_workers=2,
#             shuffle=False
#         )

In [ ]:
final_output = []
jaccards = []
test_df.loc[:, "selected_text"] = test_df.text.values
test_dataset = TweetDataset(test_df, cfg.TOKENIZER, cfg.MAX_LEN)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    shuffle=False,
    batch_size=cfg.VALID_BATCH_SIZE,
    num_workers=1
)

with torch.no_grad():
    tk0 = tqdm(test_data_loader, total=len(test_data_loader))
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_text = d["orig_text"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"].numpy()

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)
        
        # get mean probability
        starts, ends = [], []
        for model in models:
            model.to(device)
            model.eval()
            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            starts.append(outputs_start)
            ends.append(outputs_end)
        
        outputs_start = torch.mean(torch.stack(starts), dim=0)
        outputs_end = torch.mean(torch.stack(ends), dim=0)
        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()

        jaccard_scores=[]
        for px, text in enumerate(orig_text):
            selected_text = orig_selected[px]
            text_sentiment = sentiment[px]
            jaccard_score, output_sentence = calculate_jaccard_score(
                original_text=text,
                target_string=selected_text,
                sentiment_val=text_sentiment,
                idx_start=np.argmax(outputs_start[px, :]),
                idx_end=np.argmax(outputs_end[px, :]),
                offsets=offsets[px]
            )
            final_output.append(output_sentence)
            jaccard_scores.append(jaccard_score)

        jaccards.append(np.mean(jaccard_scores))
        tk0.set_postfix(jaccard=np.mean(jaccards))
len(final_output)

In [ ]:
final_output

In [ ]:
def post_process(selected):
    return " ".join(set(selected.lower().split()))

sample = pd.read_csv("../input/tweet-sentiment-extraction/sample_submission.csv")
sample.loc[:, 'selected_text'] = final_output
sample.selected_text = sample.selected_text.map(post_process)
sample.to_csv("submission.csv", index=False)